In [ ]:
!pip install transformers
!pip install datasets
!pip install ipywebrtc
!pip install soundfile
!pip install pydub
!pip install ffmpeg-python
!pip install speechbrain

# Automatic Speech Recognition

In [ ]:
import torch

def get_device():
  device='cpu'
  if torch.cuda.is_available():
    device='cuda:0'
  elif torch.backends.mps.is_available():
    device='mps'
  else:
    device='cpu'
  return 'cpu'

device = get_device()
print (device)

cpu


In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier
import torchaudio
import torchaudio.transforms as T
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
from transformers import SpeechT5HifiGan


In [ ]:
model_name = "speechbrain/spkrec-xvect-voxceleb"

speaker_classifier = EncoderClassifier.from_hparams(
    source=model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", model_name)
)

def compute_speaker_embedding(audio_data):
    with torch.no_grad():
        embeddings = speaker_classifier.encode_batch(torch.tensor(audio_data))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        embeddings = embeddings.squeeze().cpu().numpy()
    return embeddings

In [ ]:
waveform, sample_rate = torchaudio.load('/Users/premtimsina/Downloads/audio_sample2.wav')

# If audio is stereo, convert to mono
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0)

# Resample the waveform to 16kHz
resampler = T.Resample(orig_freq=sample_rate, new_freq=16000)
waveform = resampler(waveform)

# Squeeze the tensor to remove the channel dimension
waveform = waveform.squeeze()


In [ ]:
speaker_emb=compute_speaker_embedding(waveform)
speaker_emb=torch.tensor(speaker_emb).reshape(-1,512)
print(speaker_emb.shape)

/var/folders/dt/v286l_1506x752dg9dwh_yl40000gn/T/ipykernel_98863/3016772246.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings = speaker_classifier.encode_batch(torch.tensor(audio_data))


torch.Size([1, 512])


In [ ]:

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [ ]:
inputs = processor(text="This is Harry. I live in New York City", return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], speaker_emb, vocoder=vocoder)

In [ ]:
from IPython.display import Audio

Audio(speech, rate=16000)